In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from skimage.feature import peak_local_max
import mrcfile
import napari
import starfile

### Set paths to tomograms and their segmentations. Set pixel size, bin factor, peak separation.

In [2]:
eman2_directory = Path("/mnt/scratch/ribosomes/kas_k44a/eman2/").absolute()
tomo_directory = (eman2_directory / Path("mrcs/")).absolute()
segment_directory = (eman2_directory / Path("segmentations/")).absolute()

ts_list = [
    "ts_001_b10_rec",
    "ts_013_b10_rec",
    "ts_014_b10_rec",
    "ts_015_b10_rec",
    "ts_021_b10_rec",
]

tomo_bin_factor = 10
peak_separation = 10 # pixels
unbinned_pixel_size = 1.0825
print(f"Peaks will be at least {peak_separation * tomo_bin_factor * unbinned_pixel_size / 10.0} nm apart.")
ts_index = 0


Peaks will be at least 10.825 nm apart.


In [3]:
# Construct dictionary
ts_dict = {}

for segment_path in sorted(segment_directory.glob("*.mrc")):
    ts_name = segment_path.stem.split("__")[0]
    # Check if tomo should be ignored
    if ts_name in ts_list:
        ts_dict[ts_name] = {}
        tomo_path = tomo_directory / f"{ts_name}.mrc"
        if tomo_path.is_file() and segment_path.is_file():
            ts_dict[ts_name]["tomo_path"] = tomo_path
            ts_dict[ts_name]["segment_path"] = segment_path
            ts_dict[ts_name]["peaks"] = None

In [4]:
ts_dict.keys()

dict_keys(['ts_001_b10_rec', 'ts_013_b10_rec', 'ts_014_b10_rec', 'ts_015_b10_rec', 'ts_021_b10_rec'])

### Set relative and absolute peak thresholds. Run the peak_local_max calcuation for each segmentation.

In [5]:
# peaks with intensity above threshold_abs or threshold_rel * max_intensity are saved
threshold_rel = 0.2
threshold_abs = 0.1

border_size_in_px = 10
#print(border_size)
peak_files = segment_directory.glob("*.peaks")

to_load = ts_dict.keys()

for ts_name in to_load:
    if Path(f"{ts_name}_{threshold_abs}abs.peaks").absolute() not in peak_files:
        ts_dict[ts_name]["tomogram_mrc"] = mrcfile.read(ts_dict[ts_name]["tomo_path"])
        ts_dict[ts_name]["segment_mrc"]= mrcfile.read(ts_dict[ts_name]["segment_path"])
        print("Finding peaks in", ts_name, "...")
        peaks = peak_local_max(
            ts_dict[ts_name]["segment_mrc"],
            min_distance = peak_separation, 
            threshold_abs = threshold_abs, 
            threshold_rel = threshold_rel,
            exclude_border = border_size_in_px,
            )
        print("Found", len(peaks), "peaks.")
        ts_dict[ts_name]["peaks"] = peaks
        #np.savetxt(segment_directory / f"{ts_name}_abs{threshold_abs}rel{threshold_rel}.peaks", peaks, delimiter="\t", fmt="%d")
    ts_index += 1


Finding peaks in ts_001_b10_rec ...
Found 247 peaks.
Finding peaks in ts_013_b10_rec ...
Found 351 peaks.
Finding peaks in ts_014_b10_rec ...
Found 507 peaks.
Finding peaks in ts_015_b10_rec ...
Found 577 peaks.
Finding peaks in ts_021_b10_rec ...
Found 393 peaks.


### View each tomogram, segmentation, and peak positions in Napari

In [ ]:
viewer = napari.Viewer(ndisplay=3)
for ts_name in to_load:
    viewer.add_image(
        ts_dict[ts_name]["tomogram_mrc"], 
        name=f"{ts_name} tomogram",
        depiction="plane",
        blending="translucent",
        contrast_limits=[-15, 15],
        visible=False,
    )
    viewer.add_image(
        ts_dict[ts_name]["segment_mrc"],
        name=f"{ts_name} segment",
        depiction="volume",
        blending="translucent",
        colormap="gist_earth",
        visible=False,
    )
    viewer.add_points(
        ts_dict[ts_name]["peaks"], 
        name=f"{ts_name} peaks",
        blending="translucent",
        edge_color="red",
        edge_width=0.2, 
        visible=False
    )


Rendering frames...


100%|██████████| 61/61 [00:05<00:00, 11.56it/s]


Rendering frames...


100%|██████████| 61/61 [00:23<00:00,  2.64it/s]


Rendering frames...


100%|██████████| 61/61 [00:22<00:00,  2.66it/s]


Rendering frames...


100%|██████████| 61/61 [00:19<00:00,  3.18it/s]


Save the peak positions in a star file for each tomogram.

In [ ]:
# Convert the calculated peak coordinates from binned space to unbinned space. Star files for RELION required unbinned coordinates.
rln_coordinates = ["rlnCoordinateZ", "rlnCoordinateY", "rlnCoordinateX"]
rln_angles = ["rlnAngleRot", "rlnAngleTilt", "rlnAnglePsi"]

#rln_tomoname = "ts002"
#peak_coords_df = pd.DataFrame(peak_coords,columns=rln_coordinates)
#unbinned_peak_coords_df = pd.DataFrame()
#for coordinate in rln_coordinates:
#    unbinned_peak_coords_df[coordinate] = tomo_bin_factor * peak_coords_df[coordinate]
#for angle in rln_angles:
#    unbinned_peak_coords_df[angle] = 0.0
#unbinned_peak_coords_df["rlnTomoName"] = rln_tomoname
#unbinned_peak_coords_df.head(3)
#
# Create empty pd columns and append entries for every tomogram in tomos
name_i = 0
unbinned_peak_coords_dict = {}

import numpy as np

for ts_name in to_load:
    short_ts_name = ts_name.split("_b")[0]
    unbinned_peak_coords_dict[ts_name] = pd.DataFrame(ts_dict[ts_name]["peaks"] * tomo_bin_factor,columns=rln_coordinates)
    for angle in rln_angles:
        random_angles = np.random.randint(0,179,size=len(unbinned_peak_coords_dict[ts_name].index))
        unbinned_peak_coords_dict[ts_name][angle] = random_angles
    unbinned_peak_coords_dict[ts_name]["rlnTomoName"] = short_ts_name
    name_i+=1

    px_shift = -10
    unbinned_peak_coords_dict[ts_name]["rlnCoordinateX"] = unbinned_peak_coords_dict[ts_name]["rlnCoordinateX"] + px_shift * tomo_bin_factor
    write_path = segment_directory / f"particles_{short_ts_name}_abs{threshold_abs}rel{threshold_rel}_Xshift{px_shift}.star"
    output_star_df = {}
    optics_columns = [
        "rlnOpticsGroup",
        "rlnOpticsGroupName",
        "rlnSphericalAberration",
        "rlnVoltage",
        "rlnTomoTiltSeriesPixelSize",
    ]
    output_star_df["optics"] = pd.DataFrame(columns=optics_columns) 
    output_star_df["optics"].loc[len(output_star_df["optics"])] = [1, "opticsGroup1", 2.7, 300, 1.0825]
    output_star_df["particles"] = unbinned_peak_coords_dict[ts_name]
    output_star_df["particles"]["rlnOpticsGroup"] = 1
    output_star_df["particles"]["rlnOriginXAngst"] = 0.0
    output_star_df["particles"]["rlnOriginYAngst"] = 0.0
    output_star_df["particles"]["rlnOriginZAngst"] = 0.0


    starfile.write(output_star_df, write_path, overwrite=True)

### Save all peak positions in all tomograms to a single star file.

In [ ]:
# Initialize the df that will be written to the star file
output_star_df = {}
output_star_df["optics"] = pd.DataFrame(columns=optics_columns)
output_star_df["optics"].loc[len(output_star_df["optics"])] = [1, "opticsGroup1", 2.7, 300, 1.0825]
output_star_df["particles"] = pd.DataFrame(columns=rln_coordinates+rln_angles+["rlnTomoName"])
output_star_df["particles"]["rlnOpticsGroup"] = 1
output_star_df["particles"]["rlnOriginXAngst"] = 0.0
output_star_df["particles"]["rlnOriginYAngst"] = 0.0
output_star_df["particles"]["rlnOriginZAngst"] = 0.0

# Concatenate particle dfs from each tomogram into one
for ts_name in unbinned_peak_coords_dict.keys():
    output_star_df["particles"]= pd.concat([output_star_df["particles"], unbinned_peak_coords_dict[ts_name]], ignore_index=True)

# Write the star file
write_path = segment_directory / f"particles_abs{threshold_abs}rel{threshold_rel}.star"
starfile.write(output_star_df, write_path, overwrite=True)
